In [1]:
from networks import ConvNeXt1DStageConfig, ConvNeXt1D, MonthlyEnergyCNN, ConvNet, Conv1DStageConfig
import torch
from torchsummary import summary

Couldn't import dot_parser, loading of dot files will not be possible.


In [2]:
b = 35
c = 128
l = 8760
k = 10

pooler = torch.nn.AdaptiveAvgPool1d(12)
linear = torch.nn.Linear(128, 64)
conv = torch.nn.Conv1d(
    in_channels=c,
    out_channels=2*c,
    kernel_size=k,
    padding='same',
    stride=1,
)

input = torch.randn((b,c,l))
print(input.shape)
conv(input).shape
for param in conv.parameters():
    print(param.numel())
print(c*2*c)

torch.Size([35, 128, 8760])
327680
256
32768


c:\Users\Sam Wolk\.conda\envs\ml-for-bem-training\lib\site-packages\torch\nn\modules\conv.py:309: UserWarning: Using padding='same' with even kernel lengths and odd dilation may require a zero-padded copy of the input be created (Triggered internally at C:\cb\pytorch_1000000000000\work\aten\src\ATen\native\Convolution.cpp:1004.)
  return F.conv1d(input, weight, bias, self.stride,


In [3]:
device = 'cuda'

In [4]:
b = 35
c = 11
l = 8760
latent = 22
latent_length = 12
# net = MonthlyEnergyCNN(in_channels=c, out_channels=latent, n_feature_maps=64).to(device)
# summary(net, (c,l))
conf = Conv1DStageConfig.Base(c)
net = ConvNet(conf, latent, latent_length).to(device)
summary(net, (c,l))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv1d-1             [-1, 64, 8760]             768
       BatchNorm1d-2             [-1, 64, 8760]             128
            Conv1d-3             [-1, 64, 8760]          34,560
       BatchNorm1d-4             [-1, 64, 8760]             128
              ReLU-5             [-1, 64, 8760]               0
       Conv1DBlock-6             [-1, 64, 8760]               0
            Conv1d-7             [-1, 64, 8760]         102,464
       BatchNorm1d-8             [-1, 64, 8760]             128
              ReLU-9             [-1, 64, 8760]               0
      Conv1DBlock-10             [-1, 64, 8760]               0
           Conv1d-11             [-1, 64, 8760]          36,928
      BatchNorm1d-12             [-1, 64, 8760]             128
             ReLU-13             [-1, 64, 8760]               0
      Conv1DBlock-14             [-1, 6

In [ ]:
batch_size = 25
channel_size = 11
timeseries_size = 8760
timeseries_final_size = 12

In [ ]:
net = ConvNeXt1D(
    in_channels=channel_size,
    series_output_size=timeseries_final_size,
    stage_configs=[
        ConvNeXt1DStageConfig(64, 256, num_layers=3),
        ConvNeXt1DStageConfig(256, 512, num_layers=3),
        ConvNeXt1DStageConfig(512, 256, num_layers=9),
        ConvNeXt1DStageConfig(256, 64, num_layers=3),
    ]
).to(device)
summary(net, input_size=(channel_size, timeseries_size))

In [ ]:
batch = torch.randn((batch_size, channel_size, timeseries_size)).to(device)

In [ ]:
y = net(batch)
print(y.shape)
l = torch.mean(y*y)
l.backward()